In [ ]:
!pip install customtkinter opencv-python-headless pillow numpy


In [ ]:
!pip install opencv-python


In [ ]:
import tkinter as tk
import customtkinter as ctk
from PIL import Image, ImageTk, ImageDraw
import threading
import time
import numpy as np

# ======================================================
# GLOBAL VARIABLES
# ======================================================
anime_img = None
anime_tk = None
image_refs = []

blink_state = 1.0
mouth_state = 0.0
expression = "neutral"

emoji_var = None   # For emoji label


# ======================================================
# ANIME FACE DRAWING
# ======================================================
def create_anime_face():
    global anime_img

    img = Image.new("RGBA", (300, 300), (250, 230, 255))
    d = ImageDraw.Draw(img)

    # Base face
    d.ellipse((40, 30, 260, 260), fill=(255, 210, 240))

    # Eyes (with blinking)
    eye_h = int(40 * blink_state)
    d.ellipse((80, 90, 140, 90 + eye_h), fill="white")
    d.ellipse((160, 90, 220, 90 + eye_h), fill="white")

    # Pupils only if eyes open
    if eye_h > 5:
        d.ellipse((100, 120, 120, 140), fill="black")
        d.ellipse((180, 120, 200, 140), fill="black")

    # Mouth (with talking)
    m_h = int(20 + mouth_state * 35)
    d.rectangle((120, 180, 180, 180 + m_h), fill=(220, 60, 90))

    # Expression details
    if expression == "happy":
        d.arc((60, 80, 130, 140), 0, 180, fill="pink", width=3)
        d.arc((170, 80, 240, 140), 0, 180, fill="pink", width=3)

    elif expression == "angry":
        d.line((75, 95, 125, 70), fill="red", width=3)
        d.line((175, 70, 225, 95), fill="red", width=3)

    anime_img = img


# ======================================================
# UPDATE ANIME FACE PANEL
# ======================================================
def update_anime_panel():
    global anime_tk

    create_anime_face()

    img_resized = anime_img.resize((300, 300))
    anime_tk = ImageTk.PhotoImage(img_resized)
    image_refs.append(anime_tk)

    anime_label.configure(image=anime_tk)
    anime_label.image = anime_tk

    # Update emoji indicator
    if expression == "happy":
        emoji_var.set("😊 Happy Mode")
    elif expression == "angry":
        emoji_var.set("😠 Angry Mode")
    else:
        emoji_var.set("😐 Neutral Mode")


# ======================================================
# BLINK LOOP
# ======================================================
def blink_loop():
    global blink_state
    while True:
        time.sleep(np.random.uniform(2, 4))
        for v in [0.8, 0.4, 0.1, 0.0]:
            blink_state = v
            update_anime_panel()
            time.sleep(0.05)
        for v in [0.1, 0.4, 0.8, 1.0]:
            blink_state = v
            update_anime_panel()
            time.sleep(0.05)


# ======================================================
# SPEAK (MOUTH ANIMATION)
# ======================================================
def speak_animation():
    global mouth_state
    for _ in range(15):
        mouth_state = np.random.uniform(0.2, 1.0)
        update_anime_panel()
        time.sleep(0.1)
    mouth_state = 0.0
    update_anime_panel()


# ======================================================
# CHANGE EXPRESSION
# ======================================================
def set_expression(exp):
    global expression
    expression = exp
    update_anime_panel()


# ======================================================
# GUI WINDOW
# ======================================================
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.title("Anime Robot Face + Emoji Indicator")
app.geometry("500x600")

ctk.CTkLabel(app, text="Anime Robot Face Panel", font=("Arial", 22)).pack(pady=10)

anime_label = tk.Label(app)
anime_label.pack()

# EMOJI LABEL (AUTO-UPDATES)
emoji_var = tk.StringVar()
ctk.CTkLabel(app, textvariable=emoji_var, font=("Arial", 20)).pack(pady=10)


# Expression buttons
btn = ctk.CTkFrame(app)
btn.pack(pady=10)

ctk.CTkButton(btn, text="😊 Happy", command=lambda: set_expression("happy")).grid(row=0, column=0, padx=5)
ctk.CTkButton(btn, text="😐 Neutral", command=lambda: set_expression("neutral")).grid(row=0, column=1, padx=5)
ctk.CTkButton(btn, text="😠 Angry", command=lambda: set_expression("angry")).grid(row=0, column=2, padx=5)

ctk.CTkButton(app, text="Speak", command=lambda: threading.Thread(target=speak_animation, daemon=True).start()).pack(pady=10)

# Start blinking thread
threading.Thread(target=blink_loop, daemon=True).start()

update_anime_panel()
app.mainloop()
